In [1]:
%%bash

curl -LO http://www.robots.ox.ac.uk/~vgg/data/pets/data/images.tar.gz
tar zxvf images.tar.gz
mv images pet_images

images/
images/boxer_16.jpg
images/chihuahua_165.jpg
images/pug_183.jpg
images/english_setter_1.jpg
images/chihuahua_170.jpg
images/english_cocker_spaniel_17.jpg
images/samoyed_39.jpg
images/Egyptian_Mau_62.jpg
images/samoyed_36.jpg
images/german_shorthaired_3.jpg
images/Ragdoll_183.jpg
images/British_Shorthair_64.jpg
images/american_pit_bull_terrier_57.jpg
images/beagle_120.jpg
images/american_bulldog_174.jpg
images/chihuahua_101.jpg
images/shiba_inu_136.jpg
images/Abyssinian_136.jpg
images/Siamese_201.jpg
images/Abyssinian_85.jpg
images/saint_bernard_145.jpg
images/Siamese_63.jpg
images/leonberger_164.jpg
images/Maine_Coon_126.jpg
images/samoyed_51.jpg
images/Birman_15.jpg
images/english_cocker_spaniel_181.jpg
images/english_cocker_spaniel_128.jpg
images/leonberger_133.jpg
images/english_cocker_spaniel_6.jpg
images/miniature_pinscher_119.jpg
images/american_pit_bull_terrier_27.jpg
images/Abyssinian_37.jpg
images/Bombay_91.jpg
images/Egyptian_Mau_6.jpg
images/Maine_Coon_173.jpg
images

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  755M  100  755M    0     0  4415k      0  0:02:55  0:02:55 --:--:-- 4793k


In [2]:
from tensorflow import keras
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, GlobalAveragePooling2D
from tensorflow.keras import backend as K
import os
import numpy as np
import json
from collections import Counter
from tensorflow.keras.optimizers import SGD

In [3]:
pet_images_fn = [fn for fn in os.listdir('pet_images') if fn.endswith('.jpg')]
labels = []
idx_to_labels = []
label_to_idx = {}
for fn in pet_images_fn:
    label, _ = fn.rsplit('_', 1)
    if not label in label_to_idx:
        label_to_idx[label] = len(idx_to_labels)
        idx_to_labels.append(label)
    labels.append(label_to_idx[label])
len(idx_to_labels)

37

In [4]:
def fetch_pet(pet):
    img = image.load_img('pet_images/' + pet, target_size=(299, 299))
    return image.img_to_array(img)

img_vector = np.asarray([fetch_pet(pet) for pet in pet_images_fn])

In [5]:
base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(299, 299, 3))
for layer in base_model.layers:
    layer.trainable = False

pool_2d = GlobalAveragePooling2D(name='pool_2d')(base_model.output)
# dense = Dense(1024, name='dense', activation='relu')(pool_2d)
dense = Dense(1024, activation='relu')(pool_2d)
predictions = Dense(len(idx_to_labels), activation='softmax')(dense)
model = Model(inputs=base_model.input, outputs=predictions)
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

87916544/87910968 [==============================] - 26s 0us/step
Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 299, 299, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 149, 149, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 149, 149, 32) 96          conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 149, 149, 32) 0           batch_normalization[0][0]        
____________________________

In [6]:
# 0.9448
y = np.zeros((len(labels), len(idx_to_labels)))
for idx, label in enumerate(labels):
    y[idx][label] = 1

model.fit(
    img_vector, y,
    batch_size=128,
    epochs=15,
    verbose=2
)

Epoch 1/15
7390/7390 - 597s - loss: 1.5842 - accuracy: 0.6525
Epoch 2/15
7390/7390 - 599s - loss: 0.5496 - accuracy: 0.8345
Epoch 3/15
7390/7390 - 601s - loss: 0.4061 - accuracy: 0.8744
Epoch 4/15
7390/7390 - 601s - loss: 0.3312 - accuracy: 0.8907
Epoch 5/15
7390/7390 - 602s - loss: 0.2721 - accuracy: 0.9088
Epoch 6/15
7390/7390 - 602s - loss: 0.2297 - accuracy: 0.9245
Epoch 7/15
7390/7390 - 603s - loss: 0.1896 - accuracy: 0.9368
Epoch 8/15
7390/7390 - 603s - loss: 0.1877 - accuracy: 0.9368
Epoch 9/15
7390/7390 - 603s - loss: 0.1528 - accuracy: 0.9498
Epoch 10/15
7390/7390 - 603s - loss: 0.1430 - accuracy: 0.9502
Epoch 11/15
7390/7390 - 604s - loss: 0.1301 - accuracy: 0.9540
Epoch 12/15
7390/7390 - 601s - loss: 0.1266 - accuracy: 0.9578
Epoch 13/15
7390/7390 - 603s - loss: 0.0885 - accuracy: 0.9709
Epoch 14/15
7390/7390 - 603s - loss: 0.0907 - accuracy: 0.9709
Epoch 15/15
7390/7390 - 604s - loss: 0.0848 - accuracy: 0.9736


In [7]:
unfreeze = False
for layer in base_model.layers:
    if unfreeze:
        layer.trainable = True
    if layer.name == 'mixed9':
        unfreeze = True
model.compile(optimizer=SGD(lr=0.0001, momentum=0.9), 
              loss='categorical_crossentropy', metrics=['accuracy'])

In [8]:
model.fit(
    img_vector, y,
    batch_size=128,
    epochs=10,
    verbose=2
)

Epoch 1/10
7390/7390 - 681s - loss: 0.0757 - accuracy: 0.9739
Epoch 2/10
7390/7390 - 679s - loss: 0.0389 - accuracy: 0.9863
Epoch 3/10
7390/7390 - 680s - loss: 0.0289 - accuracy: 0.9903
Epoch 4/10
7390/7390 - 680s - loss: 0.0256 - accuracy: 0.9928
Epoch 5/10
7390/7390 - 679s - loss: 0.0222 - accuracy: 0.9939
Epoch 6/10
7390/7390 - 679s - loss: 0.0201 - accuracy: 0.9954
Epoch 7/10
7390/7390 - 680s - loss: 0.0193 - accuracy: 0.9951
Epoch 8/10
7390/7390 - 679s - loss: 0.0199 - accuracy: 0.9949
Epoch 9/10
7390/7390 - 682s - loss: 0.0184 - accuracy: 0.9958
Epoch 10/10
7390/7390 - 684s - loss: 0.0181 - accuracy: 0.9955


In [9]:
json.dump(idx_to_labels, open('zoo/09.3 pet_labels.json', 'w'))
model.save('zoo/09.3 retrained pet recognizer.h5')
